# Alternative Tracing Methods

So far in this module, we've taken a look at the traceable decorator, and how we can use it to set up tracing.

In this lesson, we're going to look at alternative ways in which we can set up tracing, and when you should think about using these different approaches.

## LangChain and LangGraph

If we are using LangChain or LangGraph, all we need to do to set up tracing is to set a few environment variables

In [1]:
from dotenv import load_dotenv
load_dotenv()  

import os
print("LangSmith Key Set:", os.getenv("LANGCHAIN_API_KEY") is not None)


LangSmith Key Set: True


Don't worry too much about our graph implementation here, you can learn more about LangGraph through our LangGraph Academy course!

In [2]:
import nest_asyncio
import operator
from langchain.schema import Document
from langchain_core.messages import HumanMessage, AnyMessage, get_buffer_string
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display
from typing import List
from typing_extensions import TypedDict, Annotated
from utils import get_vector_db_retriever, RAG_PROMPT

nest_asyncio.apply()

retriever = get_vector_db_retriever()
llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# Define Graph state
class GraphState(TypedDict):
    question: str
    messages: Annotated[List[AnyMessage], operator.add]
    documents: List[Document]

# Define Nodes
def retrieve_documents(state: GraphState):
    messages = state.get("messages", [])
    question = state["question"]
    documents = retriever.invoke(f"{get_buffer_string(messages)} {question}")
    return {"documents": documents}

def generate_response(state: GraphState):
    question = state["question"]
    messages = state["messages"]
    documents = state["documents"]
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    
    rag_prompt_formatted = RAG_PROMPT.format(context=formatted_docs, conversation=messages, question=question)
    generation = llm.invoke([HumanMessage(content=rag_prompt_formatted)])
    return {"documents": documents, "messages": [HumanMessage(question), generation]}

# Define Graph
graph_builder = StateGraph(GraphState)
graph_builder.add_node("retrieve_documents", retrieve_documents)
graph_builder.add_node("generate_response", generate_response)
graph_builder.add_edge(START, "retrieve_documents")
graph_builder.add_edge("retrieve_documents", "generate_response")
graph_builder.add_edge("generate_response", END)

simple_rag_graph = graph_builder.compile()
display(Image(simple_rag_graph.get_graph().draw_mermaid_png()))

We're setting up a simple graph in LangGraph. If you want to learn more about LangGraph, I would highly recommend taking a look at our LangGraph Academy course.

You can also pass in metadata or other fields through an optional config

In [3]:
question = "How do I set up tracing if I'm using LangChain?"
simple_rag_graph.invoke({"question": question}, config={"metadata": {"foo": "bar"}})

##### Let's take a look in LangSmith!

## Tracing Context Manager

In Python, you can use the trace context manager to log traces to LangSmith. This is useful in situations where:

You want to log traces for a specific block of code.
You want control over the inputs, outputs, and other attributes of the trace.
It is not feasible to use a decorator or wrapper.
Any or all of the above.
The context manager integrates seamlessly with the traceable decorator and wrap_openai wrapper, so you can use them together in the same application.

You still need to set your `LANGSMITH_API_KEY` and `LANGSMITH_TRACING`

![AWTT](../../images/alternative_ways_to_trace_2.png)

In [12]:
from langsmith import trace  # import trace context manager

def retrieve_documents(question: str):
    with trace(
        name="Retrieve Documents",
        run_type="retriever",
        inputs={"question": question},
        metadata={"provider": MODEL_PROVIDER}
    ) as ls_trace:
        documents = retriever.invoke(question)
        ls_trace.end(outputs={"documents_count": len(documents)})
        return documents

def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)

    with trace(
        name="Generate Response",
        run_type="chain",
        inputs={"question": question, "formatted_docs": formatted_docs},
        metadata={"provider": MODEL_PROVIDER, "model": MODEL_NAME},
    ) as ls_trace:

        messages = [
            {
                "role": "system",
                "content": RAG_SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": f"Context: {formatted_docs} \n\n Question: {question}"
            }
        ]

        response = call_groq(messages)
        # Assuming response is an AIMessage with .content attribute
        ls_trace.end(outputs={"output": response.content})
        return response

def call_groq(messages: List[dict], model: str = MODEL_NAME, temperature: float = 0.0):
    with trace(
        name="Call Groq LLM",
        run_type="llm",
        inputs={"messages": messages, "model": model, "temperature": temperature},
        metadata={"provider": MODEL_PROVIDER}
    ) as ls_trace:
        response = openai_client.invoke(input=messages)
        # Assuming response is an AIMessage with .content attribute
        ls_trace.end(outputs={"output": response.content})
        return response

def langsmith_rag(question: str):
    with trace(
        name="LangSmith RAG Pipeline",
        run_type="pipeline",
        inputs={"question": question},
        metadata={"provider": MODEL_PROVIDER, "model": MODEL_NAME, "app_version": APP_VERSION}
    ) as ls_trace:

        documents = retrieve_documents(question)
        response = generate_response(question, documents)

        ls_trace.end(outputs={"final_response": response.content})
        return response.content


In [15]:
question = "Can you explain the alternative ways of tracing?"
ai_answer = langsmith_rag(question)
print(ai_answer)

There are alternative ways to implement tracing, including using OpenTelemetry, which allows you to think of a LangSmith trace as a collection of spans. Additionally, LangSmith supports distributed tracing out of the box, linking runs within a trace across services using context propagation headers. You can also use manual instrumentation and configuration to set up tracing in your application.


## wrap_openai

The wrap_openai/wrapOpenAI methods in Python/TypeScript allow you to wrap your OpenAI client in order to automatically log traces -- no decorator or function wrapping required! The wrapper works seamlessly with the @traceable decorator or traceable function and you can use both in the same application.

You still need to set your `LANGSMITH_API_KEY` and `LANGSMITH_TRACING`

![AWTT](../../images/alternative_ways_to_trace_3.png)

In [17]:
from langsmith import traceable, trace
from langchain_groq import ChatGroq
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

MODEL_PROVIDER = "groq"
MODEL_NAME = "llama-3.3-70b-versatile"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

# Initialize Groq LLM client
groq_client = ChatGroq(model=MODEL_NAME, temperature=0.0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

@traceable(run_type="chain")
def retrieve_documents(question: str):
    # Make sure retriever returns list of objects with `page_content` attribute
    docs = retriever.invoke(question)
    # Just in case docs is None or empty, return empty list
    if docs is None:
        return []
    return docs

def generate_response(question: str, documents: List):
    # Safely join page_content strings from documents
    formatted_docs = "\n\n".join(getattr(doc, "page_content", "") for doc in documents)

    with trace(
        name="Generate Response",
        run_type="chain",
        inputs={"question": question, "formatted_docs": formatted_docs},
        metadata={"provider": MODEL_PROVIDER, "model": MODEL_NAME},
    ) as ls_trace:

        messages = [
            {
                "role": "system",
                "content": RAG_SYSTEM_PROMPT
            },
            {
                "role": "user",
                "content": f"Context: {formatted_docs} \n\n Question: {question}"
            }
        ]

        response = call_groq(messages)
        ls_trace.end(outputs={"output": response})

    return response

@traceable(run_type="llm")
def call_groq(
    messages: List[dict],
):
    # Groq client expects messages as input
    response = groq_client.invoke(input=messages)
    return response

@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)

    # Groq returns AIMessage object, access content directly
    return response.content

if __name__ == "__main__":
    question = "How do I trace with tracing context?"
    answer = langsmith_rag(question)
    print("Answer:", answer)


Answer: To trace with tracing context, you can use the `tracing_context` context manager in Python, which allows you to enable or disable tracing for specific parts of your application. You can do this by wrapping the code you want to trace with `with ls.tracing_context(enabled=True):`. In JavaScript/TypeScript, you can pass a `LangChainTracer` instance as a callback to achieve similar functionality.


The wrapped OpenAI client accepts all the same langsmith_extra parameters as @traceable decorated functions

In [22]:
from langsmith import trace
from langchain_groq import ChatGroq

MODEL_NAME = "llama-3.3-70b-versatile"
groq_client = ChatGroq(model=MODEL_NAME, temperature=0.0)

messages = [
    {"role": "user", "content": "What are the migratory patterns of cranes?"}
]

with trace(
    name="Groq Chat Completion",
    run_type="llm",
    metadata={"foo": "bar"},
) as ls_trace:
    response = groq_client.invoke(input=messages)
    ls_trace.end(outputs={"response": response.content})

print(response.content)


Cranes are known for their impressive migratory patterns, with many species traveling thousands of miles each year. Here are some general patterns and interesting facts about crane migration:

**Migration Routes:**

1. **Siberian Cranes**: Migrate from their breeding grounds in Siberia to their wintering grounds in India, China, and Southeast Asia, a journey of over 4,000 miles (6,400 km).
2. **Sandhill Cranes**: Migrate from their breeding grounds in Canada and the northern United States to their wintering grounds in the southern United States, Mexico, and the Gulf Coast, a journey of up to 3,000 miles (4,800 km).
3. **Whooping Cranes**: Migrate from their breeding grounds in Canada to their wintering grounds in Texas, a journey of over 2,500 miles (4,000 km).
4. **Common Cranes**: Migrate from their breeding grounds in Europe and Asia to their wintering grounds in Africa, the Middle East, and southern Europe, a journey of up to 3,000 miles (4,800 km).

**Migration Patterns:**

1. **F

## [Advanced] RunTree

Another, more explicit way to log traces to LangSmith is via the RunTree API. This API allows you more control over your tracing - you can manually create runs and children runs to assemble your trace. You still need to set your `LANGSMITH_API_KEY`, but `LANGSMITH_TRACING` is not necessary for this method.

![AWTT](../../images/alternative_ways_to_trace_4.png)

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGSMITH_API_KEY"] = ""
os.environ["LANGSMITH_PROJECT"] = "langsmith-academy"

In [ ]:
from dotenv import load_dotenv
# I have my env variables defined in a .env file
load_dotenv(dotenv_path="../../.env", override=True)

Let's go ahead and set `LANGSMITH_TRACING` to false, as we are using RunTree to manually create runs in this case.

In [ ]:
import os
os.environ["LANGSMITH_TRACING"] = "false"

from langsmith import utils
utils.tracing_is_enabled() # This should return false

We have rewritten our RAG application, except this time we pass a RunTree argument through our function calls, and create child runs at each layer. This gives our RunTree the same hierarchy that we were automatically able to establish with @traceable

In [ ]:
from langsmith import RunTree
from openai import OpenAI
from typing import List
import nest_asyncio
from utils import get_vector_db_retriever

openai_client = OpenAI()
nest_asyncio.apply()
retriever = get_vector_db_retriever()

def retrieve_documents(parent_run: RunTree, question: str):
    # Create a child run
    child_run = parent_run.create_child(
        name="Retrieve Documents",
        run_type="retriever",
        inputs={"question": question},
    )
    documents = retriever.invoke(question)
    # Post the output of our child run
    child_run.end(outputs={"documents": documents})
    child_run.post()
    return documents

def generate_response(parent_run: RunTree, question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    rag_system_prompt = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the latest question in the conversation. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.
    """
    # Create a child run
    child_run = parent_run.create_child(
        name="Generate Response",
        run_type="chain",
        inputs={"question": question, "documents": documents},
    )
    messages = [
        {
            "role": "system",
            "content": rag_system_prompt
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    openai_response = call_openai(child_run, messages)
    # Post the output of our child run
    child_run.end(outputs={"openai_response": openai_response})
    child_run.post()
    return openai_response

def call_openai(
    parent_run: RunTree, messages: List[dict], model: str = "gpt-4o-mini", temperature: float = 0.0
) -> str:
    # Create a child run
    child_run = parent_run.create_child(
        name="OpenAI Call",
        run_type="llm",
        inputs={"messages": messages},
    )
    openai_response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    # Post the output of our child run
    child_run.end(outputs={"openai_response": openai_response})
    child_run.post()
    return openai_response

def langsmith_rag(question: str):
    # Create a root RunTree
    root_run_tree = RunTree(
        name="Chat Pipeline",
        run_type="chain",
        inputs={"question": question}
    )

    # Pass our RunTree into the nested function calls
    documents = retrieve_documents(root_run_tree, question)
    response = generate_response(root_run_tree, question, documents)
    output = response.choices[0].message.content

    # Post our final output
    root_run_tree.end(outputs={"generation": output})
    root_run_tree.post()
    return output
    


In [ ]:
question = "How can I trace with RunTree?"
ai_answer = langsmith_rag(question)
print(ai_answer)